In [ ]:
# !pip install wandb

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 13.6 MB/s 
     |████████████████████████████████| 7.6 MB 66.7 MB/s 
     |████████████████████████████████| 182 kB 62.4 MB/s 


In [ ]:
# import pandas as pd

# df = pd.read_excel("/content/final_dataset_clean_v2.xlsx",skiprows=1)

In [48]:
# df.answer = df.answer.apply(str)

In [49]:
# import pandas as pd

# def df_to_squad(df):
#     # Initialize an empty list to store the documents
#     documents = []

#     # Group the dataframe by document
#     for name, group in df.groupby("id"):
#         # Initialize a dictionary to store the document
#         document = {
#             "id": name,
#             "title": group["title"].iloc[0],
#             "paragraphs": []
#         }

#         # Group the data by paragraph
#         for _, paragraph_group in group.groupby("id"):
#             # Initialize a dictionary to store the paragraph
#             paragraph = {
#                 "context": paragraph_group["paragraph"].iloc[0],
#                 "qas": []
#             }

#             # Iterate over the questions and answers in the paragraph
#             for _, row in paragraph_group.iterrows():
#                 # Initialize a dictionary to store the question-answer pair
#                 qa = {
#                     "id": row["id"],
#                     "question": row["question"],
#                     "answers": [
#                         {
#                             "text": row["answer"],
#                             "answer_start": row["answer_start"]
#                         }
#                     ]
#                 }
#                 # Add the question-answer pair to the paragraph
#                 paragraph["qas"].append(qa)

#             # Add the paragraph to the document
#             document["paragraphs"].append(paragraph)

#         # Add the document to the list of documents
#         documents.append(document)

#     return {"data":documents}

In [50]:
# data = df_to_squad(df)

In [51]:
# import json
# with open("/content/data.json", "w") as f:
#     json.dump(data, f)

In [52]:
# from sklearn.model_selection import train_test_split

# # Split the data into training and testing sets
# train_data, test_data = train_test_split(data['data'], test_size=0.2, shuffle=True)

In [53]:
# X_train_dec = {"data":train_data}
# X_test_dec = {"data":test_data}

In [54]:
# with open("/content/train_RODI.json", "w") as f:
#     json.dump(X_train_dec, f)

# with open("/content/valid_RODI.json", "w") as f:
#     json.dump(X_test_dec, f)


In [3]:
import wandb

wandb.init(project="QA_comparison", entity="omarmohamed88")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
import json
import numpy as np

from transformers import (AutoTokenizer, 
  AutoModelForQuestionAnswering, 
  AdamW,
 get_linear_schedule_with_warmup,
 DistilBertTokenizerFast)
 

import torch 
import torch.nn as nn
from torch.utils.data import DataLoader

from tqdm import tqdm

In [5]:

# running the model on to the gpu if it is possibile for better performance

if torch.cuda.is_available():
  device = torch.device('cuda')
  print('Running on gpu')
else:
    device = torch.device('cpu')
    print('Running on cpu')

Running on gpu


In [11]:
def read_squad(path):
    # open JSON file and load intro dictionary
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    # initialize lists for contexts, questions, and answers
    contexts = []
    questions = []
    answers = []
    # iterate through all data in squad data
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                # check if we need to be extracting from 'answers' or 'plausible_answers'
                if 'plausible_answers' in qa.keys():
                    access = 'plausible_answers'
                else:
                    access = 'answers'
                for answer in qa[access]:
                    # append data to lists
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
    # return formatted data lists
    return contexts, questions, answers


def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
       
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters


def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = 511
        if end_positions[-1] is None:
            end_positions[-1] = 511

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

In [12]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)


def load_dataset(file, tokenizer, batch_size):
    contexts, questions, answers = read_squad(file)
    add_end_idx(answers, contexts)
    encodings = tokenizer(contexts, questions, truncation=True, padding=True, max_length=512, return_tensors="pt")
    add_token_positions(encodings, answers)

    dataset = SquadDataset(encodings)

    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return loader


def load_datasets(train_file, valid_file, test_file, tokenizer, batch_size):
    train_loader = load_dataset(train_file, tokenizer, batch_size)
    valid_loader = load_dataset(valid_file, tokenizer, 1)
    test_loader = load_dataset(test_file, tokenizer, 1)
    return train_loader ,valid_loader ,test_loader

In [8]:
tokenizer = AutoTokenizer.from_pretrained('racai/distilbert-multi-base-romanian-cased')

model = AutoModelForQuestionAnswering.from_pretrained("racai/distilbert-multi-base-romanian-cased")


Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/534 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/397k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/328M [00:00<?, ?B/s]

Some weights of the model checkpoint at racai/distilbert-multi-base-romanian-cased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at racai/distilbert-multi-base-romanian-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias

In [13]:
import re
import string
import torch


def normalize_text(s):
    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))


def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()

    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)

    common_tokens = set(pred_tokens) & set(truth_tokens)

    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0

    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)

    return 2 * (prec * rec) / (prec + rec)


def get_gold_answers(example):
    """helper function that retrieves all possible true answers from a squad2.0 example"""

    gold_answers = [answer["text"] for answer in example.answers if answer["text"]]

    # if gold_answers doesn't exist it's because this is a negative example -
    # the only correct answer is an empty string
    if not gold_answers:
        gold_answers = [""]

    return gold_answers


def compute_scores(inputs, start_logits, end_logits, gold_start, gold_end, tokenizer):
    answer_start = torch.argmax(start_logits)
    answer_end = torch.argmax(end_logits) + 1

    prediction = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs[0][answer_start:answer_end]))
    gold_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs[0][gold_start:gold_end + 1]))

    # print(f"Prediction: {prediction}")
    # print(f"True Answers: {gold_answer}")

    em_score = compute_exact_match(prediction, gold_answer)
    f1_score = compute_f1(prediction, gold_answer)

    # print(f"EM: {em_score} \t F1: {f1_score}")

    return em_score, f1_score

In [15]:
epochs = 25
batch_size = 12
learning_rate = 3e-5


train_loader, valid_loader, _ = load_datasets("/content/train_RODI.json",
                                              "/content/valid_RODI.json",
                                              "/content/valid_RODI.json",
                                              tokenizer,
                                              12)

# move model over to detected device
model.to(device)
wandb.watch(model)

# activate training mode of model
model.train()
# initialize adam optimizer with weight decay (reduces chance of overfitting)
optim = AdamW(model.parameters(), lr=learning_rate)

best_f1 = -1
scheduler = get_linear_schedule_with_warmup(optim,
                                            num_warmup_steps=len(train_loader),
                                            num_training_steps=len(train_loader) * epochs)

for epoch in range(epochs):
    # set model to train mode
    model.train()
    total_loss = 0
    # setup loop (we use tqdm for the progress bar)
    loop = tqdm(train_loader, leave=True)
    for i, batch in enumerate(loop):
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all the tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        # train model on batch and return outputs (incl. loss)
        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)

        # extract loss
        loss = outputs[0]
        total_loss += loss.item()
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        scheduler.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=total_loss / (i + 1))

    total_em = 0
    total_f1 = 0
    model.eval()
    loop = tqdm(valid_loader, leave=True)
    for i, batch in enumerate(loop):
        # pull all the tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        # train model on batch and return outputs (incl. loss)
        outputs = model(input_ids, attention_mask=attention_mask)
        # extract loss
        em_score, f1_score = compute_scores(input_ids,
                                            outputs.start_logits, outputs.end_logits,
                                            batch["start_positions"], batch["end_positions"],
                                            tokenizer)

        total_em += em_score
        total_f1 += f1_score

        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}, EM {total_em / (i + 1)}, F1 {total_f1 / (i + 1)}')
        wandb.log({"Epoch": epoch})
        wandb.log({"Excat match": total_em / (i + 1)})
        wandb.log({"F1 score": total_f1 / (i + 1)})
   
    if total_f1 / len(loop) > best_f1:
        print(f"Best checkpoint found: {best_f1} -> {total_f1 / len(loop)}")
        torch.save(model, "model.pt")
        best_f1 = total_f1 / len(loop)

Epoch 0: 100%|██████████| 559/559 [03:38<00:00,  2.56it/s, loss=5.58]
Epoch 0, EM 0.029832935560859187, F1 0.20723389370876072: 100%|██████████| 1676/1676 [00:31<00:00, 52.87it/s]


Best checkpoint found: -1 -> 0.20723389370876072


Epoch 1: 100%|██████████| 559/559 [03:40<00:00,  2.54it/s, loss=4.23]
Epoch 1, EM 0.04295942720763723, F1 0.2194032478899844: 100%|██████████| 1676/1676 [00:34<00:00, 48.48it/s]


Best checkpoint found: 0.20723389370876072 -> 0.2194032478899844


Epoch 2: 100%|██████████| 559/559 [03:40<00:00,  2.53it/s, loss=2.96]
Epoch 2, EM 0.1652744630071599, F1 0.36396726838655713: 100%|██████████| 1676/1676 [00:31<00:00, 53.66it/s]


...

Epoch 9: 100%|██████████| 559/559 [03:40<00:00,  2.53it/s, loss=0.611]
Epoch 9, EM 0.2607398568019093, F1 0.4831207568172373: 100%|██████████| 1676/1676 [00:30<00:00, 54.70it/s]


Best checkpoint found: 0.48215639481096434 -> 0.4831207568172373


Epoch 10: 100%|██████████| 559/559 [03:40<00:00,  2.53it/s, loss=0.533]
Epoch 10, EM 0.2535799522673031, F1 0.4738077866768777: 100%|██████████| 1676/1676 [00:31<00:00, 54.01it/s]
Epoch 11: 100%|██████████| 559/559 [03:40<00:00,  2.54it/s, loss=0.455]
Epoch 11, EM 0.25178997613365156, F1 0.4743864025776593: 100%|██████████| 1676/1676 [00:31<00:00, 53.66it/s]
Epoch 12: 100%|██████████| 559/559 [03:40<00:00,  2.53it/s, loss=0.405]
Epoch 12, EM 0.2541766109785203, F1 0.48411215665430113: 100%|██████████| 1676/1676 [00:31<00:00, 53.80it/s]


Best checkpoint found: 0.4831207568172373 -> 0.48411215665430113


Epoch 13: 100%|██████████| 559/559 [03:40<00:00,  2.54it/s, loss=0.36]
Epoch 13, EM 0.25238663484486873, F1 0.47793904998259296: 100%|██████████| 1676/1676 [00:31<00:00, 53.86it/s]
Epoch 14: 100%|██████████| 559/559 [03:40<00:00,  2.53it/s, loss=0.325]
Epoch 14, EM 0.2571599045346062, F1 0.4759486990397487: 100%|██████████| 1676/1676 [00:31<00:00, 53.23it/s]
Epoch 15: 100%|██████████| 559/559 [03:40<00:00,  2.54it/s, loss=0.276]
Epoch 15, EM 0.24701670644391407, F1 0.47670055709823106: 100%|██████████| 1676/1676 [00:31<00:00, 53.56it/s]
Epoch 16: 100%|██████████| 559/559 [03:40<00:00,  2.54it/s, loss=0.25]
Epoch 16, EM 0.2583532219570406, F1 0.4774423342663875: 100%|██████████| 1676/1676 [00:31<00:00, 52.80it/s]
Epoch 17: 100%|██████████| 559/559 [03:40<00:00,  2.53it/s, loss=0.228]
Epoch 17, EM 0.25238663484486873, F1 0.47670187202611414: 100%|██████████| 1676/1676 [00:30<00:00, 54.38it/s]
Epoch 18: 100%|██████████| 559/559 [03:40<00:00,  2.54it/s, loss=0.211]
Epoch 18, EM 0.248210023

In [17]:
model = torch.load("model.pt", map_location=device)
model.to(device)

_, _, test_loader = load_datasets("/content/train_RODI.json",
                                  "/content/valid_RODI.json",
                                  "/content/valid_RODI.json",
                                  tokenizer,
                                  1)

total_em = 0
total_f1 = 0
model.eval()
loop = tqdm(test_loader, leave=True)
for i, batch in enumerate(loop):
    # pull all the tensor batches required for training
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    # train model on batch and return outputs (incl. loss)
    outputs = model(input_ids, attention_mask=attention_mask)
    # extract loss
    em_score, f1_score = compute_scores(input_ids,
                                        outputs.start_logits, outputs.end_logits,
                                        batch["start_positions"], batch["end_positions"],
                                        tokenizer)

    total_em += em_score
    total_f1 += f1_score

    # print relevant info to progress bar
    loop.set_description(f'EM {total_em / (i + 1)}, F1 {total_f1 / (i + 1)}')

print(f'Excat Match {total_em / (i + 1)}, F1 score {total_f1 / (i + 1)}')
     

EM 0.2541766109785203, F1 0.4841121566543009: 100%|██████████| 1676/1676 [00:28<00:00, 59.38it/s]


Excat Match 0.2541766109785203, F1 score 0.4841121566543009
